In [1]:
import random

In [33]:
class State:
    def __init__(self, pieces=None, enemy_pieces=None):
        self.pieces = pieces if pieces != None else [0] * 9
        self.enemy_pieces = enemy_pieces if enemy_pieces != None else [0] * 9
        
    def piece_count(self, pieces):
        count = 0
        for i in pieces:
            if i == 1:
                count += 1
        
        return count
    
    def is_lose(self):
        def is_comp(x, y, dx, dy):
            """３並びかどうか"""
            for k in range(3):
                if y < 0 or 2 < y or x < 0 or 2 < x or self.enemy_pieces[x + y * 3] == 0:
                    return False
                x, y = x + dx, y + dy
            
            return True
        
        if is_comp(0, 0, 1, 1) or is_comp(0, 2, 1, -1):
            return True
        
        for i in range(3):
            if is_comp(0, i, 1, 0) or is_comp(i, 0, 0, 1):
                return True
            
        return False
    
    def is_draw(self):
        return self.piece_count(self.pieces) + self.piece_count(self.enemy_pieces) == 9
    
    def is_done(self):
        return self.is_lose() or self.is_draw()
    
    def next(self, action):
        pieces = self.pieces.copy()
        pieces[action] = 1
        return State(self.enemy_pieces, pieces)
    
    def legal_actions(self):
        """空欄のインデックスを返す"""
        actions = []
        for i in range(9):
            if self.pieces[i] == 0 and self.enemy_pieces[i] == 0:
                actions.append(i)
                
        return actions
    
    def is_first_player(self):
        return self.piece_count(self.pieces) == self.piece_count(self.enemy_pieces)
    
    def __str__(self):
        ox = ('o', 'x') if self.is_first_player() else ('x', 'o')
        str = ''
        for i in range(9):
            if self.pieces[i] == 1:
                str += ox[0]
            elif self.enemy_pieces[i] == 1:
                str += ox[1]
            else:
                str += '-'
            if i % 3 == 2:
                str += '\n'
                
        return str

In [34]:
def random_action(state):
    legal_actions = state.legal_actions()
    return legal_actions[random.randint(0, len(legal_actions) - 1)]

In [35]:
state = State()

while True:
    if state.is_done():
        break
        
    action = random_action(state)
    state = state.next(action)
    
    print(state)

---
--o
---

-x-
--o
---

-x-
--o
--o

-x-
-xo
--o

-x-
oxo
--o

-x-
oxo
x-o

-xo
oxo
x-o



In [36]:
def mini_max(state):
    if state.is_lose():
        return -1
    if state.is_draw():
        return 0
    
    best_score = -float('inf')
    for action in state.legal_actions():
        score = -mini_max(state.next(action))
        best_score = max(best_score, score)
        
    return best_score

In [44]:
def mini_max_action(state):
    best_action = 0
    best_score = -float('inf')
    str = ['', '']
    for action in state.legal_actions():
        score = -mini_max(state.next(action))
        if score > best_score:
            best_action = action
            best_score = score
            
        str[0] = '{}{:2d},'.format(str[0], action)
        str[1] = '{}{:2d},'.format(str[1], score)
        
    print('action: {}\nscore: {}'.format(str[0], str[1]))
    
    return best_action

In [45]:
state = State()

while True:
    if state.is_done():
        break
    
    if state.is_first_player():
        action = mini_max_action(state)
    else:
        action = random_action(state)
        
    state = state.next(action)
    
    print(state)

action:  0, 1, 2, 3, 4, 5, 6, 7, 8,
score:  0, 0, 0, 0, 0, 0, 0, 0, 0,
o--
---
---

o-x
---
---

action:  1, 3, 4, 5, 6, 7, 8,
score: -1, 1, 0, 0, 1, 0, 1,
o-x
o--
---

o-x
o--
x--

action:  1, 4, 5, 7, 8,
score: -1, 1,-1,-1,-1,
o-x
oo-
x--

o-x
oo-
x-x

action:  1, 5, 7,
score: -1, 1,-1,
o-x
ooo
x-x

